In [2]:
# %% [code]
# Import necessary libraries
import json      # For reading JSON files
import os        # For file path operations
import pandas as pd   # For data manipulation and analysis
import datetime  # For converting timestamps to human-readable dates


In [4]:
# Define the path using ~
data_filepath = "/Users/tim/astrasart/crypto-signal/src/data/bitcoin_data.json"

# Expand the ~ to the full path (e.g., /Users/tim)
data_filepath = os.path.expanduser(data_filepath)

# Open and load the JSON file
with open(data_filepath, "r") as f:
    crypto_data = json.load(f)



In [6]:
ls -l /Users/tim/astrasart/crypto-signal/src/data/bitcoin_data.json


-rw-r--r--  1 tim  staff  169368 Feb  4 00:37 /Users/tim/astrasart/crypto-signal/src/data/bitcoin_data.json


In [8]:
import os
print(os.getcwd())


/Users/tim/astrasart/crypto-signal/notebooks


In [10]:
# -------------------------------
# Step 2: Extract the price data
# -------------------------------
def extract_price_data(data):
    """
    Extracts timestamp and price data from the loaded JSON data.
    The JSON is expected to have a 'prices' key containing a list of [timestamp, price] pairs.
    
    Args:
        data (dict): The cryptocurrency data loaded from JSON.
    
    Returns:
        list: A list of tuples in the form (timestamp, price).
    """
    prices = data.get("prices", [])
    return [(item[0], item[1]) for item in prices]

# Use the function to get the price data
price_data = extract_price_data(crypto_data)


In [12]:
# -------------------------------
# Step 3: Create a Pandas DataFrame from the price data
# -------------------------------
# Create the DataFrame with columns for timestamp and price
df = pd.DataFrame(price_data, columns=['timestamp', 'price'])

# Convert the timestamp (which is in milliseconds) to a datetime object
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

# Sort the DataFrame by datetime (if not already sorted)
df.sort_values('datetime', inplace=True)


In [14]:
# -------------------------------
# Step 4: Calculate a simple moving average
# -------------------------------
# Set the window size for the moving average (you can adjust this value)
window_size = 5

# Use Pandas' rolling() method to calculate the moving average of the 'price' column
df['moving_average'] = df['price'].rolling(window=window_size).mean()


In [16]:
# -------------------------------
# Step 5: Calculate the raw signal
# -------------------------------
# The raw signal is defined as the difference between the current price and its moving average.
df['raw_signal'] = df['price'] - df['moving_average']

In [26]:
# -------------------------------
# Step 6: Define Drake's formula with tunable parameters
# -------------------------------
def drakes_formula(signal_strength, num_factors, normalization_factor):
    """
    Applies a modified version of Drake's formula to the raw signal.
    The formula multiplies the raw signal by a weight and then normalizes it.
    
    Args:
        signal_strength (float): The raw signal (price difference).
        num_factors (float): A parameter that influences the weight multiplier.
                             (For example, num_factors=1 produces a multiplier of 1.1.)
        normalization_factor (float): A value to scale down the weighted signal.
    
    Returns:
        float: The normalized weighted signal.
    """
    # Calculate the weight multiplier (this is a simple linear scaling)
    drake_weight = 2.0 + (num_factors * 0.2)
    
    # Multiply the raw signal by the weight multiplier
    weighted_signal = signal_strength * drake_weight
    
    # Normalize the weighted signal to bring it closer to a desired range (e.g., -1 to 1)
    normalized_signal = weighted_signal / normalization_factor
    
    return normalized_signal


In [28]:
# -------------------------------
# Step 7: Set the tunable parameters for Drake's formula
# -------------------------------
# You can adjust these values to see how the normalized signal changes.
num_factors = 1           # For example, try 1, 1.5, 2, etc.
normalization_factor = 500.0  # Adjust based on your desired scaling


In [30]:
# -------------------------------
# Step 8: Calculate the weighted and normalized signals
# -------------------------------
# Apply the Drake's formula to each raw signal in the DataFrame
df['weighted_signal'] = df['raw_signal'] * (1.0 + (num_factors * 0.1))
df['normalized_signal'] = df['weighted_signal'] / normalization_factor


In [32]:
# -------------------------------
# Step 9: Output the results in a Pandas DataFrame
# -------------------------------
# Create a summary DataFrame with relevant columns for easy copying
df_summary = df[['datetime', 'price', 'moving_average', 'raw_signal', 'weighted_signal', 'normalized_signal']]

# Display the first few rows to inspect the results
print(df_summary.head())

# Alternatively, you can display the entire DataFrame (or export it to CSV)
# df_summary.to_csv("parameter_tuning_results.csv", index=False)

                 datetime         price  moving_average  raw_signal  \
0 2025-01-05 06:00:58.589  98332.785558             NaN         NaN   
1 2025-01-05 07:03:49.334  98432.366367             NaN         NaN   
2 2025-01-05 08:03:10.339  98245.812482             NaN         NaN   
3 2025-01-05 09:03:56.425  97817.764425             NaN         NaN   
4 2025-01-05 10:03:54.704  97637.687070     98093.28318 -455.596111   

   weighted_signal  normalized_signal  
0              NaN                NaN  
1              NaN                NaN  
2              NaN                NaN  
3              NaN                NaN  
4      -501.155722          -1.002311  
